<a href="https://colab.research.google.com/github/NucleiDatorum/Portfolio/blob/main/subscriber_loyalty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='DarkBlue' size=+2><b>Исследование потребительской лояльности крупной российской телеком-компании</b></font><br>

Ссылка на дашборд в Tableau:<br>
https://public.tableau.com/views/_NPS-/NPS-?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

<font color='DarkBlue' size=+1>Описание проекта</font><br>
Крупная телекоммуникационная компания, оказывающая услуги на территории всего СНГ, <br>
провела опрос среди своих клиентов из России, задав им классический вопрос: <br>
"Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым".

<font color='DarkBlue' size=+1>Цель исследования</font><br>
Необходимо определить текущий уровень потребительской лояльности (NPS) <br>
в зависимости от пользовательских признаков <br>
и подготовить итоговый дашборд.<br>
Результаты опроса следует разделить на три группы:<br>
- 9-10 баллов - "сторонники";
- 7-8 баллов - "нейтралы";
- 0-6 баллов - "критики".<br>

Итоговое значение NPS рассчитать по формуле: % "сторонников" - % "критиков".

<font color='DarkBlue' size=+1>Описание данных</font><br>
База данных с результатами опроса хранится в файле СУБД SQLite.<br>
В ней 5 таблиц:
- <font color='blue'> user </font> содержит основную информацию о клиентах;
- <font color='blue'> location </font> - это справочник территорий, в которых оказывает услуги телеком-компания;
- <font color='blue'> age_segment </font> - данные о возрастных сегментах клиентов;
- <font color='blue'> traffic_segment </font> - данные о сегментах по объёму потребляемого трафика;
- <font color='blue'> lifetime_segment </font> - данные о сегментах по количеству месяцев "жизни" клиента (лайфтайм - это показатель, определяющий, является ли клиент новым. Клиента считают новым, если количество дней "жизни" в компании составляет не более 365 дней).

## 1. Подключение к базе данных

In [1]:
# импорт необходимых библиотек

import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# путь к БД на локальном диске
path_to_db_local = '/content/telecomm_csi.db'

# путь к БД на платформе Яндекс Практикум
path_to_db_platform = '/datasets/telecomm_csi.db'

# итоговый путь к БД
path_to_db = None

# если путь на локальном диске ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local

# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform

# иначе вывести сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой, то создать подключение к БД
if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## 2. Витрина данных для дашборда

### 2.1. SQL-запрос

Чтобы подготовить данные для дашборда, необходимо собрать их в одну витрину при помощи SQL-запросов. Нам нужны не все данные из базы, а только следующие поля:
- <font color='blue'> user_id </font> - идентификатор клиента;
- <font color='blue'> lt_day </font> - количество дней "жизни" клиента;
- <font color='blue'> is_new </font> - является ли клиент новым (этого поля нет в исходной БД, его нужно создать при помощи подзапроса CASE, который разделит клиентов на категории "new" и "old");
- <font color='blue'> age </font> - возраст клиента;
- <font color='blue'> gender_segment </font> - пол (в БД это поле содержит значения: 1 - женщина, 0 - мужчина. Для удобства работы это поле следует преобразовать в значения: "мужчина" и "женщина" при помощи подзапроса CASE);
- <font color='blue'> os_name </font> - тип операционной системы;
- <font color='blue'> cpe_type_name </font> - тип устройства;
- <font color='blue'> country </font> - страна проживания;
- <font color='blue'> city </font> - город проживания;
- <font color='blue'> age_segment </font> - возрастной сегмент (в исходной базе данных вместе с возрастом указан двухзначный код сегмента. Чтобы выгрузить только возраст, воспользуемся оператором SUBSTRING);
- <font color='blue'> traffic_segment </font> - сегмент по объёму потребляемого трафика (выгрузим только трафик, без двухзначного кода);
- <font color='blue'> lifetime_segment </font> - сегмент по количеству дней "жизни" (выгрузим только количество дней);
- <font color='blue'> nps_score </font> - оценка клиента в NPS-опросе (от 1 до 10);
- <font color='blue'> nps_grop </font> - группа, к которой относится оценка клиента (новое поле, которое нужно создать при помощи подзапроса CASE, разделив оценки на категории: "сторонники", "нейтралы" и "критики").

Для объединения данных из разных таблиц используем запрос LEFT JOIN.

In [3]:
# SQL-запрос
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'new'
           ELSE 'old'
       END is_new,
       u.age,
       CASE
           WHEN u.gender_segment == 1 THEN 'женщина'
           WHEN u.gender_segment == 0 THEN 'мужчина'
           ELSE 'неизвестно'
       END gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTRING (ags.title, 3) AS age_segment,
       SUBSTRING (ts.title, 3) AS traffic_segment,
       SUBSTRING (ls.title, 3) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score >= 9 THEN 'сторонники'
           WHEN u.nps_score <= 8 AND nps_score >= 7 THEN 'нейтралы'
           WHEN u.nps_score <= 6 THEN 'критики'
       END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS ags ON ags.age_gr_id = u.age_gr_id
LEFT JOIN traffic_segment AS ts ON ts.tr_gr_id = u.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON ls.lt_gr_id = u.lt_gr_id;
"""

### 2.2. Датафрейм

In [4]:
# сохраним полученные данные в датафрейм df
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,old,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,old,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


### 2.3. Файл в формате .csv

In [5]:
# полученную витрину сохраним в формате .csv
df.to_csv('telecomm_csi_tableau.csv', index=False)

## 3. Дашборд

Из полученных данных средствами программы Tableau Public составлен интерактивный дашборд, <br>
в котором отражен уровень лояльности в зависимости от пользовательских признаков, <br>
таких как: пол, возраст, город проживания, операционная система и т.д.

Ссылка на дашборд в Tableau:
https://public.tableau.com/views/_NPS-/NPS-?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link